<table align="left">
  <td>
    <a href="https://tinyurl.com/2khrsfbc" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

# Natural Language Processing with TensorFlow